In [ ]:
# ========================================
# CIFAR-10 CNN Classifier with Data Augmentation
# and Visualization of Predictions
# ========================================

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import random

# 1️⃣ Load CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # normalize

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Original training samples: {x_train.shape[0]}")
print(f"Test samples: {x_test.shape[0]}")

# 2️⃣ Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    shear_range=0.1
)
datagen.fit(x_train)

# Data augmentation doesn’t *increase* the dataset size on disk,
# but generates new images in memory during each epoch.
# To demonstrate, let’s “generate” one epoch of augmented data:
augmented_images, _ = next(datagen.flow(x_train, y_train, batch_size=len(x_train), shuffle=False))
print(f"Augmented images generated in memory: {augmented_images.shape[0]}")

# 3️⃣ Build CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 4️⃣ Train Model with Data Augmentation
batch_size = 64
epochs = 20

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train)//batch_size,
    epochs=epochs,
    verbose=1
)

# 5️⃣ Evaluate Model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc*100:.2f}%")

# 6️⃣ Plot Accuracy & Loss Curves
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 7️⃣ Predict and Visualize Results
num_images = 12
indices = random.sample(range(len(x_test)), num_images)
images = x_test[indices]
true_labels = y_test[indices].flatten()

predictions = model.predict(images)
predicted_labels = np.argmax(predictions, axis=1)

plt.figure(figsize=(12,8))
for i in range(num_images):
    plt.subplot(3,4,i+1)
    plt.imshow(images[i])
    color = 'green' if predicted_labels[i] == true_labels[i] else 'red'
    plt.title(f"P: {class_names[predicted_labels[i]]}\nT: {class_names[true_labels[i]]}", color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 1296s 8us/step
Original training samples: 50000
Test samples: 10000
Augmented images generated in memory: 50000


/home/nazmulhasan77/miniconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
E0000 00:00:1761111804.461600    7735 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1761111804.657103    7735 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,168,362 (8.27 MB)

...

 Non-trainable params: 0 (0.00 B)

/home/nazmulhasan77/miniconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 125s 157ms/step - accuracy: 0.3998 - loss: 1.6381 - val_accuracy: 0.5521 - val_loss: 1.2293
Epoch 2/20
  1/781 ━━━━━━━━━━━━━━━━━━━━ 1:44 134ms/step - accuracy: 0.5000 - loss: 1.5430

/home/nazmulhasan77/miniconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


781/781 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5000 - loss: 1.5430 - val_accuracy: 0.5617 - val_loss: 1.1984
Epoch 3/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 119s 152ms/step - accuracy: 0.5458 - loss: 1.2663 - val_accuracy: 0.6337 - val_loss: 1.0430
Epoch 4/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6719 - loss: 1.0628 - val_accuracy: 0.6312 - val_loss: 1.0470
Epoch 5/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 116s 149ms/step - accuracy: 0.6003 - loss: 1.1295 - val_accuracy: 0.6675 - val_loss: 0.9554
Epoch 6/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5469 - loss: 1.1872 - val_accuracy: 0.6678 - val_loss: 0.9512
Epoch 7/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 119s 152ms/step - accuracy: 0.6274 - loss: 1.0536 - val_accuracy: 0.6861 - val_loss: 0.8941
Epoch 8/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7031 - loss: 0.9245 - val_accuracy: 0.6939 - val_loss: 0.8746
Epoch 9/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 121s 154ms/step - accuracy: 0.6494 - loss: 0.9942 - val_accurac